In [1]:
# 📦 Install necessary libraries if not already installed (for Colab)
!pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:

from transformers import pipeline, set_seed
import torch
import re
import textwrap

# Optional: Set seed for reproducibility
set_seed(42)

# === Step 1: Model Selection with Dropdown-like Menu ===
available_models = {
    '1': ('distilgpt2', 'DistilGPT-2 (Lightweight, ~82MB)'),
    '2': ('gpt2', 'GPT-2 (Small, ~500MB)'),
    '3': ('gpt2-medium', 'GPT-2 Medium (~1.5GB)'),
    '4': ('gpt2-large', 'GPT-2 Large (~3GB)'),
    '5': ('EleutherAI/gpt-neo-1.3B', 'GPT-Neo 1.3B (~5GB)'),
    '6': ('EleutherAI/gpt-neo-2.7B', 'GPT-Neo 2.7B (~10GB)'),
    '7': ('microsoft/DialoGPT-medium', 'DialoGPT Medium (~800MB)')
}

print("🤖 Available Models:")
print("=" * 50)
for key, (model_id, description) in available_models.items():
    print(f"{key}. {description}")
print("=" * 50)

while True:
    choice = input("Select a model (1-7): ").strip()
    if choice in available_models:
        model_id, model_description = available_models[choice]
        print(f"✅ Selected: {model_description}")
        break
    else:
        print("❌ Invalid choice. Please select 1-7.")

# === Step 2: Input Topic ===
prompt = input("\n📝 Enter your topic or prompt: ")

# === Step 3: Device Setup ===
device = 0 if torch.cuda.is_available() else -1
print(f"\n🚀 Loading model `{model_id}` on {'CUDA' if device == 0 else 'CPU'}...")

# === Step 4: Load Model and Generate Text ===
try:
    generator = pipeline("text-generation", model=model_id, device=device)

    result = generator(
        prompt,
        max_length=250,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.5,
        truncation=True,
        pad_token_id=50256
    )[0]["generated_text"]

    # === Step 5: Text Processing for Complete Sentences ===
    def ensure_complete_sentence(text):
        """Ensure text ends with a complete sentence"""
        # Find all sentence endings
        sentence_endings = re.finditer(r'[.!?]+(?:\s|$)', text)
        endings = list(sentence_endings)

        if not endings:
            # If no sentence endings found, return original text
            return text

        # Get the position of the last complete sentence
        last_ending = endings[-1].end()

        # Return text up to the last complete sentence
        complete_text = text[:last_ending].strip()
        return complete_text

    # Process the generated text
    processed_text = ensure_complete_sentence(result)

    # === Step 6: Format and Display Result ===
    print("\n" + "=" * 60)
    print("🧠 GENERATED TEXT")
    print("=" * 60)

    # Format the text with proper paragraph wrapping
    wrapped_text = textwrap.fill(
        processed_text,
        width=70,
        initial_indent="    ",
        subsequent_indent="    "
    )

    print(f"\n📋 Model Used: {model_description}")
    print(f"📏 Length: {len(processed_text)} characters")
    print(f"📝 Word Count: {len(processed_text.split())} words")
    print("\n" + "-" * 60)
    print("📖 FORMATTED OUTPUT:")
    print("-" * 60)
    print(wrapped_text)
    print("\n" + "=" * 60)
    print("✨ Generation Complete!")

except Exception as e:
    print(f"❌ Error loading model or generating text: {e}")
    print("💡 Try selecting a smaller model if you're running out of memory.")

🤖 Available Models:
1. DistilGPT-2 (Lightweight, ~82MB)
2. GPT-2 (Small, ~500MB)
3. GPT-2 Medium (~1.5GB)
4. GPT-2 Large (~3GB)
5. GPT-Neo 1.3B (~5GB)
6. GPT-Neo 2.7B (~10GB)
7. DialoGPT Medium (~800MB)
Select a model (1-7): 6
✅ Selected: GPT-Neo 2.7B (~10GB)

📝 Enter your topic or prompt: The Impact of AI on Humans

🚀 Loading model `EleutherAI/gpt-neo-2.7B` on CUDA...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🧠 GENERATED TEXT

📋 Model Used: GPT-Neo 2.7B (~10GB)
📏 Length: 1215 characters
📝 Word Count: 221 words

------------------------------------------------------------
📖 FORMATTED OUTPUT:
------------------------------------------------------------
    The Impact of AI on Humans  There are a lot of people out there
    who believe that artificial intelligence is going to be the end of
    mankind. This belief seems to be supported by a few people who are
    willing to admit that they don’t know everything about AI but have
    some strong opinions about it. In this article, I want to address
    a few questions that will help us to understand the impact of AI
    on humanity.  Before I get into the discussion, I want to clarify
    something: I am not a “humanity optimist.” I believe that humanity
    will always have problems and that we will always face challenges.
    It’s just that many of these problems are not because of
    technological advances. What I am trying to do here is t